# Appendix

---

# Appendix A: Data Construction & Harmonization

### A.1 The "Dual-Pipeline" Ingestion Strategy
A significant barrier to longitudinal analysis of US banking data is the format change that occurred in 2001. Standard merging procedures often fail to bridge this gap, resulting in "short panels" that drop the 1980s and 90s. To resolve this, we employed two distinct extraction algorithms:

1.  **The Legacy Pipeline (1984–2000):** This pipeline handles SAS Transport (`.xpt`) files. The primary challenge in this era is inconsistent date recording; dates are often omitted from the file content or stored in non-standard "days since 1960" formats. Our algorithm recovers the valid reporting quarter by parsing the original filenames (e.g., identifying `8206` as June 1982) and cross-validating with internal file metadata.
    
2.  **The Modern Pipeline (2001–2019):** This pipeline handles the modern bulk repository, which is organized into nested directories of variable-delimited text files. The algorithm recursively scans these directories, extracts the `Schedule RC` (Balance Sheet) and `Schedule RI` (Income Statement) files, and merges them on the unique bank identifier (`IDRSSD`).

### A.2 Variable Harmonization
Both pipelines standardize variables to a common dictionary before merging. Specifically, we prioritize "Consolidated Bank" reported values (Prefix `RCFD`) and use "Domestic Office" values (Prefix `RCON`) as a fallback for smaller banks that do not report consolidated figures. This ensures that the definition of "Total Assets" and "Commercial & Industrial Loans" remains consistent across the 35-year sample.

---

# Appendix B: Replication Code

To replicate the bank dataset construction, place the raw FFIEC data folders in the root directory and execute the scripts in the following order.

**Step 1: Process Legacy Data (1984–2000)**
Run `old_bank_data.py` to parse the SAS Transport files.
* **Input:** Raw `.xpt` files (recursive search).
* **Output:** `OLD_RAW_BANK_DATA.csv`

**Step 2: Process Modern Data (2001–2019)**
Run `new_bank_scrape.py` to parse the modern text repositories.
* **Input:** FFIEC folders containing `Schedule RC.txt` and `Schedule CI.txt`.
* **Output:** `NEW_RAW_BANK_DATA.csv`

**Step 3: Merge**
Run `merge_bank.py` Combine the two outputs to form final file `ALL_BANKS_MERGED.py`

---

To replicate the macro dataset construction, place the raw excel spreadsheet in the same directory the following scripts and execute them in the following order.

**Step 1: Clean Transcripts**
Run `fomc_data.py` to parse the transcripts.
* **Input:** transcript `.xlsx` file.
* **Output:** `fomc_data` folder, containing text files with transcripts of each date.

**Step 2: Calculate Sentiment**
Run `sentiment.py` to parse the txt files in the previously created fomc_data file.
* **Input:** fomc_data folder containing `meeting_yyyy-mm-dd.txt`.
* **Output:** `fomc_sentiment_data.csv`

**Step 3: Scrape Effective Funds Rate**
Run `rate.py` to pull latest fomc data from the St. Louis Fed.
* **Input:** N/A
* **Output:** `fed_funds.csv`

---

**Merge Data Into Final Dataset**
Run `merge.py` to pull latest fomc data from the St. Louis Fed.
* **Input:** `fed_funds.csv`, `ALL_BANKS_MERGED.csv`, `fomc_sentiment_data.csv`
* **Output:** `MACRO_DATA_CLEAN.csv`, `master_merged_dataset.csv`


### B.1 Legacy Parser (`old_bank_data.py`)


In [ ]:
import pandas as pd
import os
import glob
import re
from pandas.tseries.offsets import QuarterEnd


def build_xpt_dataset_smart():
    print("--- STARTING OLD ERA (XPT) DATA BUILDER V2 (CORRECTED) ---")

    # 1. Define SAS Variable Names (Old Era)
    # RSSD9001: Entity ID
    # RSSD9999: Report Date (Standard internal date variable)

    # ASSETS:
    # RCFD2170: Total Assets (Consolidated)
    # RCON2170: Total Assets (Domestic)

    # LOANS:
    # RCFD1763: Total C&I Loans (Consolidated) - CRITICAL for large banks
    # RCFD1766: C&I Loans to U.S. Addressees (Consolidated)
    # RCON1766: C&I Loans (Domestic)

    id_col = 'RSSD9001'
    date_col_internal = 'RSSD9999'

    # We define the columns we WANT to find.
    # Note: We don't filter strictly on read because read_sas reads the whole file anyway.
    target_cols = [
        'RSSD9001', 'RSSD9999',
        'RCFD2170', 'RCON2170',
        'RCFD1763', 'RCFD1766', 'RCON1766'
    ]

    root_dir = "."
    xpt_files = glob.glob(os.path.join(
        root_dir, "**", "*.xpt"), recursive=True)

    print(f"Found {len(xpt_files)} .xpt files. Processing...")

    master_list = []

    for file_path in xpt_files:
        filename = os.path.basename(file_path)
        date_obj = None

        # --- STRATEGY 1: PARSE FILENAME (YYMM) ---
        match = re.search(
            r'([0-9]{2})([0-9]{2})\.xpt$', filename, re.IGNORECASE)

        if match:
            yy = int(match.group(1))
            mm = int(match.group(2))

            # Year Pivot Logic (Window: 1950-2049)
            if yy > 50:
                full_year = 1900 + yy
            else:
                full_year = 2000 + yy

            try:
                # Snap to QuarterEnd
                date_obj = pd.Timestamp(
                    year=full_year, month=mm, day=1) + QuarterEnd(0)
            except:
                print(f"  Warning: Invalid date in filename {filename}")

        # --- STRATEGY 2: READ AND EXTRACT ---
        try:
            # Read SAS file
            df = pd.read_sas(file_path, format='xport')

            # Convert columns to Uppercase
            df.columns = [c.upper() for c in df.columns]

            # If date not found in filename, try internal SAS date
            if date_obj is None:
                if date_col_internal in df.columns:
                    # SAS Dates are days since 1960-01-01
                    sas_date_val = df[date_col_internal].mode()[0]
                    date_obj = pd.to_datetime(
                        '1960-01-01') + pd.to_timedelta(sas_date_val, unit='D')
                    print(
                        f"  Extracted internal date {date_obj.date()} from {filename}")
                else:
                    print(f"Skipping {filename}: Could not determine date.")
                    continue

            # Standardize ID
            if id_col not in df.columns:
                continue

            df[id_col] = pd.to_numeric(df[id_col], errors='coerce')
            df.rename(columns={id_col: 'bank_id'}, inplace=True)

            # --- STANDARDIZE ASSETS ---
            # Priority: RCFD (Consolidated) > RCON (Domestic)
            # Create a base series of NaNs
            df['assets'] = pd.NA

            if 'RCON2170' in df.columns:
                df['assets'] = df['RCON2170']

            if 'RCFD2170' in df.columns:
                # combine_first: if 'RCFD' is present, use it; else keep 'RCON'
                df['assets'] = df['RCFD2170'].combine_first(df['assets'])

            # --- STANDARDIZE C&I LOANS ---
            # Priority: RCFD1763 (Total Global) > RCFD1766 (US Global) > RCON1766 (Domestic)

            # Start with Domestic as baseline
            loan_series = df.get('RCON1766', pd.Series([pd.NA]*len(df)))

            # Update with US Global if available (fills gaps or overwrites depending on preference,
            # here we want RCFD to supersede RCON if the bank reports RCFD)
            if 'RCFD1766' in df.columns:
                loan_series = df['RCFD1766'].combine_first(loan_series)

            # Update with Total Global (Best Metric)
            if 'RCFD1763' in df.columns:
                loan_series = df['RCFD1763'].combine_first(loan_series)

            df['ci_loans'] = loan_series

            # --- FINAL DATA PREP ---
            df['date'] = date_obj

            # Keep only valid data columns
            df = df[['bank_id', 'date', 'assets', 'ci_loans']]
            df = df.dropna(subset=['bank_id'])

            master_list.append(df)
            print(
                f"Processed {filename}: {len(df)} banks. Date: {date_obj.date()}")

        except Exception as e:
            print(f"  Error reading {filename}: {e}")

    # --- COMPILE ---
    if not master_list:
        print("No data found.")
        return

    print("Compiling Old Era Master Dataset...")
    full_old_df = pd.concat(master_list, ignore_index=True)

    # Final Polish
    full_old_df['assets'] = pd.to_numeric(
        full_old_df['assets'], errors='coerce')
    full_old_df['ci_loans'] = pd.to_numeric(
        full_old_df['ci_loans'], errors='coerce')

    # Remove rows with 0 assets to avoid DivideByZero
    full_old_df = full_old_df[full_old_df['assets'] > 0]

    # Calculate Risk Taking
    full_old_df['risk_taking'] = full_old_df['ci_loans'] / \
        full_old_df['assets']

    # Final cleanup of NaNs created by risk calc
    full_old_df = full_old_df.dropna(subset=['risk_taking'])

    output_filename = 'OLD_RAW_BANK_DATA.csv'
    full_old_df.to_csv(output_filename, index=False)

    print(f"\nSUCCESS! Processed {len(full_old_df)} rows from Old Era.")
    print(f"Saved to: {output_filename}")


if __name__ == "__main__":
    build_xpt_dataset_smart()

### B.2 Modern Parser (`new_bank_data.py`)

In [ ]:
import pandas as pd
import os
import glob
import re


def build_bank_dataset_recursive():
    print("--- STARTING BANK DATA BUILDER ---")

    # 1. Setup Columns
    # IDRSSD: Bank ID
    # RCFD2170: Total Assets (Consolidated)
    # RCON2170: Total Assets (Domestic - fallback)

    # LOANS:
    # RCFD1763: Total C&I Loans (Consolidated - Includes Foreign & Domestic)
    # RCFD1766: C&I Loans to U.S. Addressees (Consolidated - Partial)
    # RCON1766: C&I Loans (Domestic)

    asset_cols = ['IDRSSD', 'RCFD2170', 'RCON2170']
    # Added RCFD1763 for accurate Total C&I on global banks
    loan_cols = ['IDRSSD', 'RCFD1763', 'RCFD1766', 'RCON1766']

    all_quarters = []

    root_dir = "."
    subfolders = [f.path for f in os.scandir(root_dir) if f.is_dir()]

    print(f"Found {len(subfolders)} folders. Scanning for data files...")

    for folder in subfolders:
        # 2. Find Schedule RC (Assets) and Schedule RCCI (Loans)
        # CRITICAL FIX: The FFIEC file for loans is named "Schedule RCCI", not "Schedule CI"
        rc_files = glob.glob(os.path.join(folder, "*Schedule RC *.txt"))
        ci_files = glob.glob(os.path.join(folder, "*Schedule RCCI *.txt"))

        if not rc_files or not ci_files:
            # Silent skip or debug print if needed
            continue

        path_rc = rc_files[0]
        path_ci = ci_files[0]

        # Extract Date
        date_match = re.search(r'(\d{8})', os.path.basename(path_rc))
        if not date_match:
            print(f"Skipping {folder}: Could not determine date.")
            continue

        date_str = date_match.group(1)

        try:
            # 3. Read ASSETS (Schedule RC)
            df_rc = pd.read_csv(path_rc, sep='\t',
                                skiprows=0, low_memory=False)
            existing_asset_cols = [c for c in asset_cols if c in df_rc.columns]
            df_rc = df_rc[existing_asset_cols]

            # 4. Read LOANS (Schedule RC-C Part I)
            df_ci = pd.read_csv(path_ci, sep='\t',
                                skiprows=0, low_memory=False)
            existing_loan_cols = [c for c in loan_cols if c in df_ci.columns]
            df_ci = df_ci[existing_loan_cols]

            # 5. Merge
            if 'IDRSSD' not in df_rc.columns or 'IDRSSD' not in df_ci.columns:
                print(f"Skipping {date_str}: Missing IDRSSD column.")
                continue

            df_quarter = pd.merge(df_rc, df_ci, on='IDRSSD', how='inner')
            df_quarter['date'] = pd.to_datetime(date_str, format='%m%d%Y')

            # 6. Standardize Assets
            if 'RCFD2170' in df_quarter.columns:
                df_quarter['assets'] = df_quarter['RCFD2170'].fillna(
                    df_quarter.get('RCON2170', 0))
            elif 'RCON2170' in df_quarter.columns:
                df_quarter['assets'] = df_quarter['RCON2170']
            else:
                df_quarter['assets'] = pd.NA

            # 7. Standardize C&I Loans (Prioritize Total Global -> US Global -> Domestic)
            # RCFD1763 = Total C&I (Consolidated)
            # RCFD1766 = C&I to US Addressees (Consolidated)
            # RCON1766 = Total C&I (Domestic)

            df_quarter['ci_loans'] = 0  # Default

            if 'RCFD1763' in df_quarter.columns:
                # Primary for large banks
                df_quarter['ci_loans'] = df_quarter['RCFD1763']

            # Fill gaps where RCFD1763 might be missing but RCFD1766 exists
            if 'RCFD1766' in df_quarter.columns:
                df_quarter['ci_loans'] = df_quarter['ci_loans'].fillna(
                    df_quarter['RCFD1766'])
                # If we initialized with 0, fillna won't work on 0s, so we use logic:
                # If 1763 was missing, column is 0 (or NaN if we didn't init).
                # Better approach: Coalesce.

            # Re-doing clean coalescing logic:
            # Create a temporary series for the best available C&I data

            # Start with Domestic (RCON1766) as baseline
            temp_loans = df_quarter.get(
                'RCON1766', pd.Series([pd.NA]*len(df_quarter)))

            # Overwrite with RCFD1766 (US Addressees) if available
            if 'RCFD1766' in df_quarter.columns:
                temp_loans = df_quarter['RCFD1766'].combine_first(temp_loans)

            # Overwrite with RCFD1763 (Total Consolidated) if available - Best Metric
            if 'RCFD1763' in df_quarter.columns:
                temp_loans = df_quarter['RCFD1763'].combine_first(temp_loans)

            df_quarter['ci_loans'] = temp_loans

            # 8. Clean up
            df_quarter.rename(columns={'IDRSSD': 'bank_id'}, inplace=True)
            df_quarter = df_quarter[['bank_id', 'date', 'assets', 'ci_loans']]

            # Ensure numeric
            df_quarter['assets'] = pd.to_numeric(
                df_quarter['assets'], errors='coerce')
            df_quarter['ci_loans'] = pd.to_numeric(
                df_quarter['ci_loans'], errors='coerce')

            all_quarters.append(df_quarter)
            print(f"Processed {date_str}: {len(df_quarter)} banks.")

        except Exception as e:
            print(f"Error reading {date_str} in {folder}: {e}")

    # 9. Compile and Save
    if not all_quarters:
        print("CRITICAL: No data found. Ensure folders are named correctly (e.g. '03312008') and contain 'Schedule RC' and 'Schedule RCCI' files.")
        return

    print("Compiling Master Dataset...")
    master_df = pd.concat(all_quarters, ignore_index=True)

    # Calculate Risk Taking
    master_df['risk_taking'] = master_df['ci_loans'] / master_df['assets']

    # Filter
    master_df = master_df.dropna(subset=['assets', 'risk_taking'])
    master_df = master_df[master_df['assets'] > 0]

    output_filename = 'NEW_RAW_BANK_DATA.csv'
    master_df.to_csv(output_filename, index=False)

    print(f"\nSUCCESS! Built dataset with {len(master_df)} rows.")
    print(f"Saved to: {output_filename}")

    # Diagnostics
    count_2008 = len(master_df[master_df['date'].dt.year == 2008])
    print(f"Diagnostics: Rows found for 2008: {count_2008}")


if __name__ == "__main__":
    build_bank_dataset_recursive()

### B.3 Bank Merge (`merge_bank.py`)

In [ ]:
import pandas as pd
import os
import glob


def find_file(filename, search_path="."):
    """Recursively searches for a file in the directory tree."""
    print(f"  Searching for '{filename}'...")
    # Recursive glob search
    matches = glob.glob(os.path.join(
        search_path, "**", filename), recursive=True)
    if matches:
        print(f"  -> Found at: {matches[0]}")
        return matches[0]
    return None


def run_smart_merge():
    print("==================================================")
    print("       SMART BANK DATA MERGER                     ")
    print("==================================================")
    print(f"Current Working Directory: {os.getcwd()}\n")

    # --- STEP 1: LOCATE FILES ---
    print("Step 1: Locating Input Files...")

    path_new = find_file("NEW_RAW_BANK_DATA.csv")
    path_old = find_file("OLD_RAW_BANK_DATA.csv")

    if not path_new and not path_old:
        print("\nCRITICAL ERROR: Could not find ANY bank data files.")
        print("You must run the builder scripts first:")
        print("  1. Run 'build_bank_data.py' (for Modern data)")
        print("  2. Run 'build_old_data_v2.py' (for Old data)")
        return

    # --- STEP 2: LOAD DATA ---
    print("\nStep 2: Loading Data...")
    dfs = []

    if path_new:
        try:
            df_new = pd.read_csv(path_new)
            print(f"  -> Loaded Modern Data: {len(df_new):,} rows")
            dfs.append(df_new)
        except Exception as e:
            print(f"  -> Error reading Modern Data: {e}")

    if path_old:
        try:
            df_old = pd.read_csv(path_old)
            print(f"  -> Loaded Old Data: {len(df_old):,} rows")
            dfs.append(df_old)
        except Exception as e:
            print(f"  -> Error reading Old Data: {e}")

    # --- STEP 3: MERGE ---
    print("\nStep 3: Merging...")
    if not dfs:
        print("  -> No data loaded. Exiting.")
        return

    merged_df = pd.concat(dfs, ignore_index=True)

    # Standardize Date
    merged_df['date'] = pd.to_datetime(merged_df['date'])

    # Deduplicate (Priority to Modern Data if overlaps exist)
    merged_df = merged_df.sort_values(['date', 'bank_id'])
    before_dedup = len(merged_df)
    merged_df = merged_df.drop_duplicates(
        subset=['bank_id', 'date'], keep='last')
    print(
        f"  -> Deduplication removed {before_dedup - len(merged_df):,} duplicates.")

    # --- STEP 4: SAVE ---
    output_filename = "ALL_BANKS_MERGED.csv"
    merged_df.to_csv(output_filename, index=False)

    print("\n" + "="*50)
    print(f"SUCCESS! Created '{output_filename}'")
    print(f"Total Rows: {len(merged_df):,}")
    print(f"Location: {os.path.abspath(output_filename)}")
    print("="*50)


if __name__ == "__main__":
    run_smart_merge()

### B.4 FOMC Data Retrieval (`data_fomc.py`)

In [ ]:
#!/usr/bin/env python3
"""
Download and prepare FOMC transcripts for sentiment analysis.

- Source: Miguel Acosta, "FOMC Communications Data"
  https://www.acostamiguel.com/data/fomc_data.html
"""

import argparse
import logging
import pathlib
import sys

import requests
import pandas as pd

TRANSCRIPTS_URL = "https://www.acostamiguel.com/data/FOMC/transcripts.xlsx"


def download_file(url: str, dest: pathlib.Path, force: bool = False) -> pathlib.Path:
    """Download file with simple resume logic (skip if exists unless --force)."""
    dest = pathlib.Path(dest)
    if dest.exists() and not force:
        logging.info(
            "File %s already exists; skipping download. Use --force to re-download.",
            dest,
        )
        return dest

    dest.parent.mkdir(parents=True, exist_ok=True)
    logging.info("Downloading %s -> %s", url, dest)

    with requests.get(url, stream=True, timeout=60) as r:
        r.raise_for_status()
        tmp = dest.with_suffix(dest.suffix + ".part")
        with open(tmp, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        tmp.replace(dest)

    logging.info("Finished download.")
    return dest


def split_meetings_to_txt(df: pd.DataFrame, out_dir: pathlib.Path) -> None:
    """Group utterances by meeting date and write one text file per meeting."""
    out_dir = pathlib.Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Choose a date column and normalize to datetime (handles raw YYYYMMDD ints).
    date_col = "meeting_date" if "meeting_date" in df.columns else "date"
    if date_col not in df.columns:
        raise KeyError("Expected a 'date' or 'meeting_date' column in the transcript data.")

    dates = df[date_col]
    if pd.api.types.is_integer_dtype(dates):
        df["meeting_date"] = pd.to_datetime(dates.astype(str), format="%Y%m%d")
    else:
        df["meeting_date"] = pd.to_datetime(dates)

    # Ensure we have ordering columns for reproducible text output.
    sort_cols = [c for c in ["sequence", "n_utterance"] if c in df.columns]

    for date, g in df.groupby("meeting_date"):
        date_str = pd.to_datetime(date).strftime("%Y-%m-%d")
        g_sorted = g.sort_values(sort_cols)

        text = "\n\n".join(g_sorted["text"].astype(str))

        out_path = out_dir / f"meeting_{date_str}.txt"
        out_path.write_text(text, encoding="utf-8")
        logging.info("Wrote %s (%d utterances)", out_path.name, len(g_sorted))


def main(argv=None):
    parser = argparse.ArgumentParser(
        description="Download and preprocess FOMC transcripts for sentiment analysis."
    )
    parser.add_argument(
        "--out-dir",
        default="fomc_data",
        help="Root output directory (default: %(default)s)",
    )
    parser.add_argument(
        "--force",
        action="store_true",
        help="Force re-download of source files even if they exist.",
    )
    parser.add_argument(
        "--xlsx",
        help="Use an existing transcripts.xlsx instead of downloading.",
    )
    parser.add_argument(
        "--meetings-dir",
        default="meetings_txt_all",
        help="Subdirectory under processed/ for per-meeting text files (default: %(default)s).",
    )

    args = parser.parse_args(argv)

    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s %(levelname)s %(message)s",
    )

    out_root = pathlib.Path(args.out_dir)
    raw_dir = out_root / "raw"
    processed_dir = out_root / "processed"

    # 1. Download (or reuse) the transcripts Excel
    if args.xlsx:
        xlsx_path = pathlib.Path(args.xlsx)
        if not xlsx_path.exists():
            raise FileNotFoundError(f"Provided --xlsx path does not exist: {xlsx_path}")
        logging.info("Using existing transcripts file at %s", xlsx_path)
    else:
        xlsx_path = download_file(
            TRANSCRIPTS_URL,
            raw_dir / "fomc_transcripts.xlsx",
            force=args.force,
        )

    # 2. Read into pandas and save a CSV version
    logging.info("Reading %s", xlsx_path)
    df = pd.read_excel(xlsx_path)

    tables_dir = processed_dir / "tables"
    tables_dir.mkdir(parents=True, exist_ok=True)

    csv_path = tables_dir / "fomc_transcripts.csv"
    df.to_csv(csv_path, index=False)
    logging.info("Saved full transcripts table to %s", csv_path)

    # 3. Write one text file per meeting
    meetings_txt_dir = processed_dir / args.meetings_dir
    split_meetings_to_txt(df, meetings_txt_dir)

    logging.info("All done. Meeting-level text lives in %s", meetings_txt_dir)


if __name__ == "__main__":
    main(sys.argv[1:])


### B.5 Sentiment Analysis (`sentiment.py`)

In [ ]:
import os
import glob
import pandas as pd
import pysentiment2 as ps

# 1. Setup
# Folder containing the text meetings. We try common locations so the script works
# whether you generated files with fomc3.py (meetings_txt_all) or an earlier step (meetings_txt).
candidate_dirs = [
    'fomc_data/processed/meetings_txt_all',
    'fomc_data/processed/meetings_txt',
    # fallback to absolute path if used elsewhere
    '/Users/hfh/Downloads/meetings_txt',
]
folder_path = next(
    (d for d in candidate_dirs if os.path.isdir(d)), candidate_dirs[0])
lm = ps.LM()  # Initialize the Financial Dictionary
results = []

# 2. The Loop
# glob.glob grabs all .txt files in the folder
file_list = glob.glob(os.path.join(folder_path, "*.txt"))
if not file_list:
    raise FileNotFoundError(
        f"No .txt files found in '{folder_path}'. "
        "Check the path or run fomc3.py to generate the meeting text files."
    )

print(f"Found {len(file_list)} files. Starting analysis...")

for file_path in file_list:
    try:
        # Get filename (e.g., "meeting_1976-03-29.txt")
        filename = os.path.basename(file_path)

        # Extract Date from filename
        # Assumes format "meeting_YYYY-MM-DD.txt"
        date_str = filename.replace('meeting_', '').replace('.txt', '')

        # Read the text file
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read()

        # 3. Analyze Sentiment
        tokens = lm.tokenize(text)
        score = lm.get_score(tokens)

        # Calculate Net Sentiment: (Pos - Neg) / (Pos + Neg)
        net_sentiment = score['Polarity']

        # Store data
        results.append({
            'date': date_str,
            'net_sentiment': net_sentiment,
            'positive_count': score['Positive'],
            'negative_count': score['Negative'],
            'word_count': len(tokens),
            'filename': filename
        })

    except Exception as e:
        print(f"Error reading {filename}: {e}")

# 4. Save Results
df = pd.DataFrame(results)

# Convert 'date' column to actual datetime objects for sorting/merging later
if df.empty:
    raise ValueError("No sentiment results produced; verify the input files.")

df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

print("Analysis Complete.")
print(df.head())

# Save to CSV for the next step of your paper
output_csv = 'fomc_sentiment_data.csv'
df.to_csv(output_csv, index=False)
print(f"Saved results to {output_csv}")

### B.6 Funds Rate Retrieval (`rate.py`)

In [ ]:
import pandas as pd


def scrape_fed_funds():
    # URL for Daily Federal Funds Rate (DFF) from St. Louis Fed
    # Use 'FEDFUNDS' instead of 'DFF' if you want monthly averages
    url = "https://fred.stlouisfed.org/graph/fredgraph.csv?id=DFF"

    print(f"Downloading data from {url}...")

    try:
        # Read CSV directly from the URL
        df = pd.read_csv(url)

        # Rename columns as requested
        df.columns = ['date', 'rate']

        # Convert date column to datetime objects for filtering
        df['date'] = pd.to_datetime(df['date'])

        # Filter for dates starting from 1982-01-01
        df = df[df['date'] >= '1976-01-01']

        # Save to CSV without the index number
        output_file = 'fed_funds.csv'
        df.to_csv(output_file, index=False)

        print(f"Success! Saved {len(df)} rows to {output_file}")
        print(f"Range: {df['date'].min().date()} to {df['date'].max().date()}")

    except Exception as e:
        print(f"Error: {e}")


if __name__ == "__main__":
    scrape_fed_funds()

### B.7 Final Dataset Merge (`merge.py`)

In [ ]:
import glob
import os
from pathlib import Path

import pandas as pd


def pick_file(csvs, label):
    """Tiny helper to pick a CSV by index with a friendly label."""
    print(f"\nWhich file contains the {label}? (Enter the number)")
    try:
        idx = int(input("Selection: "))
        return csvs[idx]
    except Exception:
        print("Invalid selection.")
        return None


def build_macro_golden_source():
    print("--- MACRO + BANK DATA BUILDER ---")
    print("We need to identify your raw source files.")

    # 1. List all CSVs in the folder (and parent folder)
    csvs = glob.glob("*.csv") + glob.glob("../*.csv")

    if not csvs:
        print("CRITICAL: No CSV files found. Please move your raw Sentiment, Fed Funds, and Bank CSVs into this folder.")
        return

    print("\nAvailable CSV files:")
    for i, f in enumerate(csvs):
        print(f"[{i}] {f}")

    # 2. Ask User for Sentiment File
    sent_file = pick_file(csvs, "SENTIMENT data")
    if not sent_file:
        return

    # 3. Ask User for Fed Funds File
    print("(Enter same number if they are in one file)")
    fed_file = pick_file(csvs, "FED FUNDS RATE")
    if not fed_file:
        return

    # 4. Ask User for Bank Panel File
    bank_file = pick_file(csvs, "BANK PANEL (assets, risk_taking, etc.)")
    if not bank_file:
        return

    # 5. Process Sentiment
    print(f"\nProcessing Sentiment from {sent_file}...")
    df_sent = pd.read_csv(sent_file)

    # Auto-detect columns (Looking for 'sentiment' or similar)
    sent_col = next((c for c in df_sent.columns if 'sentiment' in c.lower()), None)
    date_col_s = next((c for c in df_sent.columns if 'date' in c.lower()), None)

    if not sent_col or not date_col_s:
        print(f"Error: Could not find 'date' or 'sentiment' column in {sent_file}")
        print(f"Columns found: {list(df_sent.columns)}")
        return

    df_sent = df_sent[[date_col_s, sent_col]].rename(
        columns={date_col_s: 'date', sent_col: 'fed_sentiment'}
    )
    df_sent['date'] = pd.to_datetime(df_sent['date'])

    # 6. Process Fed Funds
    print(f"Processing Fed Funds from {fed_file}...")
    df_fed = pd.read_csv(fed_file)

    # Auto-detect columns
    fed_col = next((c for c in df_fed.columns if 'fund' in c.lower() or 'rate' in c.lower()), None)
    date_col_f = next((c for c in df_fed.columns if 'date' in c.lower()), None)

    if not fed_col or not date_col_f:
        print(f"Error: Could not find 'date' or 'rate' column in {fed_file}")
        return

    df_fed = df_fed[[date_col_f, fed_col]].rename(
        columns={date_col_f: 'date', fed_col: 'fed_funds_rate'}
    )
    df_fed['date'] = pd.to_datetime(df_fed['date'])

    # 7. Merge & Aggregate to Quarter
    print("Merging and aggregating macro data to quarterly level...")

    macro_df = pd.merge(df_sent, df_fed, on='date', how='outer')
    macro_df['quarter_key'] = macro_df['date'].dt.to_period('Q').astype(str)

    macro_clean = (
        macro_df.groupby('quarter_key')[['fed_sentiment', 'fed_funds_rate']]
        .mean()
        .reset_index()
    )

    macro_output = 'MACRO_DATA_CLEAN.csv'
    macro_clean.to_csv(macro_output, index=False)
    print(f"\nSUCCESS! Created '{macro_output}' with {len(macro_clean)} quarters.")

    # 8. Load Bank Panel and merge with macro to build regression-ready dataset
    print(f"\nProcessing Bank Panel from {bank_file}...")
    df_bank = pd.read_csv(bank_file)

    required_bank_cols = {'bank_id', 'date', 'assets', 'risk_taking'}
    missing = required_bank_cols - set(df_bank.columns)
    if missing:
        print(f"Error: Bank file missing required columns: {missing}")
        return

    df_bank['date'] = pd.to_datetime(df_bank['date'])
    df_bank['quarter_key'] = df_bank['date'].dt.to_period('Q').astype(str)

    merged = pd.merge(df_bank, macro_clean, on='quarter_key', how='left')

    # Keep core variables regression.py expects
    cols_order = [
        'bank_id', 'date', 'assets', 'ci_loans', 'risk_taking',
        'fed_sentiment', 'fed_funds_rate'
    ]
    # ci_loans may be missing, so include if present
    cols_order = [c for c in cols_order if c in merged.columns]
    merged = merged[cols_order + [c for c in merged.columns if c not in cols_order]]

    output = 'master_merged_dataset.csv'
    merged.to_csv(output, index=False)

    # Write uppercase alias for backwards compatibility.
    Path('MASTER_MERGED_DATASET.csv').write_bytes(Path(output).read_bytes())

    print(f"\nSUCCESS! Created '{output}' with {len(merged)} rows.")
    print("Sample rows:")
    print(merged.head())


if __name__ == "__main__":
    build_macro_golden_source()


---

# Appendix C: Regression Code

In [ ]:
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

warnings.simplefilter(action='ignore', category=FutureWarning)


def run_robust_regression():
    print("--- [1] DATA PREPARATION & CLEANING ---")
    dataset_paths = [
        Path('master_merged_dataset.csv'),
        Path('MASTER_MERGED_DATASET.csv'),
    ]
    df = None
    for path in dataset_paths:
        if path.exists():
            df = pd.read_csv(path)
            print(f"Loaded dataset from {path}")
            break
    if df is None:
        print("MASTER dataset not found. Please run merge.py first.")
        return

    df['date'] = pd.to_datetime(df['date'])

    # --- ADDRESSING POINT 9: Drop Short Panels ---
    # Banks with fewer than 8 observations (2 years) do not contribute enough within-variance
    counts = df.groupby('bank_id')['date'].count()
    valid_banks = counts[counts >= 8].index
    df = df[df['bank_id'].isin(valid_banks)].copy()
    print(
        f" > Dropped short-lived entities. Banks remaining: {df['bank_id'].nunique():,}")

    # --- ADDRESSING POINT 1: Orthogonalization ---
    # Isolate pure sentiment from the rate level
    ortho_data = df[['fed_sentiment', 'fed_funds_rate']].dropna()
    X_ortho = sm.add_constant(ortho_data['fed_funds_rate'])
    model_ortho = sm.OLS(ortho_data['fed_sentiment'], X_ortho).fit()
    df.loc[ortho_data.index, 'sentiment_shock'] = model_ortho.resid

    # --- ADDRESSING POINT 2: Standardization ---
    # Standardize shock to 1 SD for interpretability
    df['sentiment_shock_std'] = df['sentiment_shock'] / \
        df['sentiment_shock'].std()

    # --- ADDRESSING POINT 4 & 10: Functional Form & Scaling ---
    # Log Assets
    df['log_assets'] = np.log(df['assets'].replace(0, np.nan))

    # CENTER Log Assets.
    # This is crucial. Now 'log_assets_centered' = 0 means "Average Sized Bank".
    # This fixes multicollinearity by orthogonalizing the interaction from the main effect.
    df['log_assets_centered'] = df['log_assets'] - df['log_assets'].mean()

    # Sort for Lagging
    df = df.sort_values(['bank_id', 'date'])

    # --- ADDRESSING POINT 5: Dynamic Structure ---
    # We MUST include lags of the dependent variable (Risk Taking)
    # We also lag controls to mitigate simultaneity (Point 8)
    vars_to_lag = ['risk_taking', 'sentiment_shock_std',
                   'fed_funds_rate', 'log_assets_centered']

    for var in vars_to_lag:
        df[f'{var}_lag'] = df.groupby('bank_id')[var].shift(1)

    # --- ADDRESSING POINT 3: Continuous Interaction ---
    # Instead of a dummy, we use Size as a continuous moderator.
    # Interaction = (Last Quarter's Shock) * (Last Quarter's Size)
    df['shock_x_size'] = df['sentiment_shock_std_lag'] * \
        df['log_assets_centered_lag']

    # --- ESTIMATION ---
    print("--- [2] ESTIMATING DYNAMIC PANEL MODEL ---")

    df = df.set_index(['bank_id', 'date'])

    reg_df = df.dropna(subset=[
        'risk_taking', 'risk_taking_lag',
        'sentiment_shock_std_lag', 'shock_x_size',
        'fed_funds_rate_lag', 'log_assets_centered_lag'
    ])

    exog_vars = [
        'const',
        'risk_taking_lag',           # Dynamics (Fixes Point 5)
        'sentiment_shock_std_lag',   # Main Effect (at mean size)
        # Interaction (Does Size change sensitivity?)
        'shock_x_size',
        'log_assets_centered_lag',   # Control for Size
        'fed_funds_rate_lag'         # Control for Rate
    ]

    reg_df['const'] = 1

    # ADDRESSING POINT 7: Fixed Effects
    mod = PanelOLS(reg_df['risk_taking'],
                   reg_df[exog_vars], entity_effects=True)

    # ADDRESSING POINT 6: Cluster Level
    # Explicitly clustering by Entity (Bank) and Time (Date)
    res = mod.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)

    print(res)

    # --- DIAGNOSTIC CHECKS ---
    print("\n--- DIAGNOSTICS ---")
    print(f"Within R-squared: {res.rsquared_within:.4f}")
    print(
        f"Autocorrelation check (LDV coeff): {res.params['risk_taking_lag']:.4f}")


if __name__ == "__main__":
    run_robust_regression()

---

# Appendix D: Graph Code

All graphs should be run with `master_merged_dataset.csv` in the same directory. 

# Figure 1: Time Series of Sentiment and Fed Funds Rate

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np

# Load your dataset
df = pd.read_csv('master_merged_dataset.csv')

# Pre-processing: Ensure date is datetime
df['date'] = pd.to_datetime(df['date'])


def plot_macro_time_series(df):
    # Aggregate to time-series level (since macro vars are constant across banks per quarter)
    macro_df = df.groupby(
        'date')[['fed_sentiment', 'fed_funds_rate']].mean().reset_index()

    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Plot Sentiment (Left Axis)
    color = 'tab:blue'
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Raw Sentiment (fed_sentiment)', color=color)
    ax1.plot(macro_df['date'], macro_df['fed_sentiment'],
             color=color, label='Fed Sentiment')
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.grid(True, linestyle='--', alpha=0.6)

    # Create a second y-axis for Fed Funds Rate
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('Fed Funds Rate (%)', color=color)
    ax2.plot(macro_df['date'], macro_df['fed_funds_rate'],
             color=color, label='Fed Funds Rate')
    ax2.tick_params(axis='y', labelcolor=color)

    plt.title('Time Series: Raw Sentiment and Fed Funds Rate',
              fontsize=14, pad=15)
    fig.tight_layout()
    plt.show()


plot_macro_time_series(df)

ModuleNotFoundError: No module named 'pandas'

## Figure 2: Data Construction Diagram (FFIEC Changes)

In [ ]:
import matplotlib.pyplot as plt


def plot_data_construction_diagram():
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis("off")

    legacy_text = (
        "Legacy FFIEC Data (Pre-2001)\n"
        "• SAS XPT fixed-width files\n"
        "• Cryptic variable codes\n"
        "• Nonstandard date strings\n"
        "• Blank missing values\n"
        "• Irregular reporting dates\n"
        "• Incomplete bank identifiers"
    )

    modern_text = (
        "Modern FFIEC Data (Post-2001)\n"
        "• CSV/TXT delimited files\n"
        "• Consistent variable labels\n"
        "• Standard quarter-end dates\n"
        "• Explicit NA markers\n"
        "• Uniform reporting structure\n"
        "• Full RSSD + cert IDs"
    )

    ax.text(
        0.05, 0.5, legacy_text,
        fontsize=11, va="center", ha="left", family="monospace"
    )

    ax.text(
        0.60, 0.5, modern_text,
        fontsize=11, va="center", ha="left", family="monospace"
    )

    # Arrow using annotate (much better scaling control)
    ax.annotate(
        "",
        xy=(0.58, 0.5),
        xytext=(0.42, 0.5),
        arrowprops=dict(
            arrowstyle="->",
            lw=2,
            mutation_scale=12,
            color="#6FA8DC"
        )
    )

    plt.tight_layout()
    plt.show()


plot_data_construction_diagram()

## Figure 3: Binscatter of Risk Taking vs. Sentiment

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np

# Load your dataset
df = pd.read_csv('master_merged_dataset.csv')

# Pre-processing: Ensure date is datetime
df['date'] = pd.to_datetime(df['date'])

def plot_binscatter_risk_sentiment(df):
    # Bin the sentiment into deciles to reduce noise
    df['sentiment_bin'] = pd.qcut(df['fed_sentiment'], q=20, labels=False)
    binned_data = df.groupby('sentiment_bin')[
        ['fed_sentiment', 'risk_taking']].mean()

    plt.figure(figsize=(10, 6))
    sns.regplot(x=binned_data['fed_sentiment'], y=binned_data['risk_taking'], scatter_kws={
                's': 100}, line_kws={'color': 'red'})

    plt.xlabel('Fed Sentiment (Binned)')
    plt.ylabel('Average Bank Risk Taking')
    plt.title('Correlation: Fed Sentiment vs. Bank Risk Taking', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.show()


plot_binscatter_risk_sentiment(df)

## Figure 4: The "Identification" Plot (Risk Taking by Bank Size)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np

# Load your dataset
df = pd.read_csv('master_merged_dataset.csv')

# Pre-processing: Ensure date is datetime
df['date'] = pd.to_datetime(df['date'])

def plot_heterogeneity_by_size(df):
    # Create size quintiles based on Assets
    df['size_quintile'] = df.groupby('date')['assets'].transform(
        lambda x: pd.qcut(x, 5, labels=['Smallest', '2', '3', '4', 'Largest']))

    # Filter to just Smallest vs Largest for clarity
    subset = df[df['size_quintile'].isin(['Smallest', 'Largest'])]

    # Aggregate over time
    time_series = subset.groupby(['date', 'size_quintile'])[
        'risk_taking'].mean().reset_index()

    plt.figure(figsize=(12, 6))
    sns.lineplot(data=time_series, x='date', y='risk_taking',
                 hue='size_quintile', style='size_quintile')

    plt.title('Risk Taking over Time: Small vs. Large Banks', fontsize=14)
    plt.ylabel('Average Risk Taking')
    plt.xlabel('Date')
    plt.legend(title='Bank Size')
    plt.grid(True, alpha=0.3)
    plt.show()


plot_heterogeneity_by_size(df)

## Figure 5: Distribution of Risk Taking (Pre/Post Crisis)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import numpy as np

# Load your dataset
df = pd.read_csv('master_merged_dataset.csv')

# Pre-processing: Ensure date is datetime
df['date'] = pd.to_datetime(df['date'])

def plot_risk_distribution(df):
    # Define periods
    df['period'] = np.where(df['date'].dt.year < 2008, 'Pre-2008', 'Post-2008')

    plt.figure(figsize=(10, 6))
    sns.kdeplot(data=df, x='risk_taking', hue='period',
                fill=True, common_norm=False, palette='crest')

    plt.title('Distribution of Bank Risk Taking: Pre vs. Post 2008', fontsize=14)
    plt.xlabel('Risk Taking Measure')
    plt.grid(True, alpha=0.3)
    plt.show()


plot_risk_distribution(df)

ModuleNotFoundError: No module named 'matplotlib'